In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import copy
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
documents = pd.read_csv('/content/drive/My Drive/datasets/words_less_than_100.csv')
dictionary = pd.read_csv('/content/drive/My Drive/datasets/dict_before_spacy.csv')
idf_all_words = pd.read_csv('/content/drive/My Drive/datasets/idf_of_all_words_fixed.csv')

In [4]:
documents = documents.drop("Unnamed: 0", 1)

In [5]:
dictionary = dictionary.drop("Unnamed: 0", 1)

In [6]:
idf_all_words = idf_all_words.drop("wordID", 1)

In [7]:
documents

,18,19,20,34,38,39,43,45,47,51,...,46409,46894,47583,48025,48093,48269,49029,49248,49393,49524
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7496,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7498,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
documents["sum"] = documents.sum(axis=1)

In [10]:
documents

,18,19,20,34,38,39,43,45,47,51,...,46894,47583,48025,48093,48269,49029,49248,49393,49524,sum
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0
7496,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0
7497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,48.0
7498,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0


In [11]:
idf_all_words

,df,idf
0,94,4.379364
1,660,2.430418
2,934,2.083182
3,272,3.316856
4,244,3.425490
...,...,...
4442,131,4.047461
4443,54,4.933674
4444,91,4.411799
4445,50,5.010635


In [12]:
tf_pog = documents.drop('sum', 1)

In [13]:
tf_pog = tf_pog.div(tf_pog.sum(axis=1), axis=0)

In [14]:
tf_pog

,18,19,20,34,38,39,43,45,47,51,...,46409,46894,47583,48025,48093,48269,49029,49248,49393,49524
0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.011364,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7496,0.0,0.0,0.031250,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7497,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.0,0.000000
7498,0.0,0.0,0.000000,0.0,0.0,0.025641,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [15]:
idf_all_words

,df,idf
0,94,4.379364
1,660,2.430418
2,934,2.083182
3,272,3.316856
4,244,3.425490
...,...,...
4442,131,4.047461
4443,54,4.933674
4444,91,4.411799
4445,50,5.010635


In [16]:
lista_kolumn = tf_pog.columns.values.tolist()


In [18]:
lista_kolumn

['18',
 '19',
 '20',
 '34',
 '38',
 '39',
 '43',
 '45',
 '47',
 '51',
 '58',
 '66',
 '70',
 '71',
 '72',
 '74',
 '76',
 '96',
 '101',
 '103',
 '107',
 '108',
 '115',
 '117',
 '118',
 '119',
 '120',
 '123',
 '129',
 '130',
 '132',
 '133',
 '134',
 '137',
 '138',
 '139',
 '142',
 '145',
 '149',
 '153',
 '159',
 '162',
 '163',
 '165',
 '167',
 '168',
 '172',
 '173',
 '174',
 '176',
 '177',
 '185',
 '187',
 '188',
 '190',
 '191',
 '193',
 '199',
 '200',
 '201',
 '202',
 '203',
 '204',
 '206',
 '207',
 '210',
 '216',
 '222',
 '227',
 '228',
 '229',
 '230',
 '231',
 '233',
 '237',
 '247',
 '254',
 '255',
 '261',
 '262',
 '263',
 '266',
 '268',
 '270',
 '283',
 '284',
 '289',
 '311',
 '312',
 '313',
 '347',
 '350',
 '353',
 '355',
 '356',
 '357',
 '359',
 '370',
 '376',
 '379',
 '381',
 '383',
 '390',
 '391',
 '395',
 '398',
 '403',
 '408',
 '412',
 '431',
 '432',
 '444',
 '464',
 '470',
 '473',
 '487',
 '495',
 '501',
 '507',
 '508',
 '525',
 '534',
 '536',
 '541',
 '543',
 '569',
 '580',
 '

In [53]:
lista_kolumn.to_csv("lista_id_slow.csv")

AttributeError: ignored

In [17]:
essunia = idf_all_words.iloc[lista_kolumn]

IndexError: ignored

In [23]:
essunia = essunia.drop("df", 1)

In [33]:
essunia

,idf
18,4.379364
19,2.430418
20,2.083182
34,3.316856
38,3.425490
...,...
48269,4.047461
49029,4.933674
49248,4.411799
49393,5.010635


In [54]:
essunia.to_csv("indeksy_pod_idf.csv")

In [23]:
idf_all_words = idf_all_words.drop('df', 1)

In [24]:
idfs = idf_all_words.values.tolist()

In [26]:
len(idfs)

4447

In [20]:
len(idfs)

4447

In [ ]:
#    tf_pog          idfs

In [29]:
tf_pog

,18,19,20,34,38,39,43,45,47,51,...,46409,46894,47583,48025,48093,48269,49029,49248,49393,49524
0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.011364,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7496,0.0,0.0,0.031250,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7497,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.0,0.000000
7498,0.0,0.0,0.000000,0.0,0.0,0.025641,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [46]:
floats = [float(x[0]) for x in idfs]

In [47]:
final_ez = tf_pog.iloc[:,:] = tf_pog.iloc[:,:]*floats

In [48]:
final_ez

,18,19,20,34,38,39,43,45,47,51,...,46409,46894,47583,48025,48093,48269,49029,49248,49393,49524
0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.023673,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7496,0.0,0.0,0.065099,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7497,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.084322,0.0,0.0,0.0,0.000000
7498,0.0,0.0,0.000000,0.0,0.0,0.108731,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [49]:
final_ez.to_csv('complete_tf_idf.csv')